In [31]:
from pybaseball import batting_stats
from pybaseball import playerid_lookup
from pybaseball import player_search_list
from pybaseball import playerid_reverse_lookup
import pandas as pd
import csv
pd.set_option('display.max_rows', 10)

In [32]:
salary_df = pd.read_csv('mlb-free-agency-2.csv')


In [33]:
salary_df = salary_df[salary_df['total_salary'].notna()]
salary_df = salary_df.drop([0])

In [34]:
# drop first column and reset index
salary_df.drop(columns=salary_df.columns[0], axis=1, inplace=True)
salary_df.reset_index(drop=True)

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink
0,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012,795,https://www.spotrac.com/redirect/player/795/
1,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012,493,https://www.spotrac.com/redirect/player/493/
2,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012,559,https://www.spotrac.com/redirect/player/559/
3,C.J. Wilson,SP,31.0,TEX,LAA,5,77500000.0,15500000.0,2012,874,https://www.spotrac.com/redirect/player/874/
4,Mark Buehrle,SP,32.7,CHW,MIA,4,58000000.0,14500000.0,2012,169,https://www.spotrac.com/redirect/player/169/
...,...,...,...,...,...,...,...,...,...,...,...
1269,Clint Frazier,OF,27.2,NYY,CHC,1,1500000.0,1500000.0,2022,23831,https://www.spotrac.com/redirect/player/23831/
1270,Johan Camargo,3B,27.9,ATL,PHI,1,1400000.0,1400000.0,2022,21154,https://www.spotrac.com/redirect/player/21154/
1271,Jhoulys Chacin,SP,33.8,COL,COL,1,1250000.0,1250000.0,2022,7307,https://www.spotrac.com/redirect/player/7307/
1272,Taylor Clarke,RP,28.5,ARI,KC,1,975000.0,975000.0,2022,17735,https://www.spotrac.com/redirect/player/17735/


In [35]:
# drop all pitchers: SP and RP
drop_SP = salary_df[salary_df['position'] == 'SP'].index
salary_df.drop(drop_SP, inplace = True)
drop_RP = salary_df[salary_df['position'] == 'RP'].index
salary_df.drop(drop_RP, inplace = True)
# replace LF,CF,RF with OF
salary_df.position.replace(['LF', 'CF', 'RF'], 'OF', inplace=True)

In [36]:
salary_df.sort_values('name', inplace=True)
salary_df.reset_index(drop=True, inplace=True)

In [37]:
# check if there's any players who share the same name
# 1. keep entires with unique ID
unique = salary_df.drop_duplicates(subset=['spotracID'])
# 2. check if there's any duplciate name in 'Unique'
unique[unique.duplicated(subset=['name'], keep=False)]
# there isn't any

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink


## Lookup fangraphs playerID using there name

In [38]:
# create playersearch DataFrame:

player_search = salary_df[['name','spotracID','spotracLink']].copy()
player_search.drop_duplicates(subset=['name'], keep='first', inplace=True)

In [39]:
for i, row in player_search.iterrows():
    player_search.at[i, 'first_name'] = row['name'].split()[0].lower()
    if len(row['name'].split()) <= 2:
        player_search.at[i, 'last_name'] = row['name'].split()[-1].lower()
    else:
        player_search.at[i, 'last_name'] = row['name'].split()[1].lower()

In [40]:
# change a.j. to 'a. j.'

for i, row in player_search.iterrows():
    if '.' in row.first_name:
        row.first_name = row.first_name.replace('.', '. ', 1)
        player_search.at[i, 'first_name'] = row.first_name
player_search

,name,spotracID,spotracLink,first_name,last_name
0,A.J. Ellis,7427,https://www.spotrac.com/redirect/player/7427/,a. j.,ellis
1,A.J. Pierzynski,188,https://www.spotrac.com/redirect/player/188/,a. j.,pierzynski
5,A.J. Pollock,10693,https://www.spotrac.com/redirect/player/10693/,a. j.,pollock
6,Aaron Hill,908,https://www.spotrac.com/redirect/player/908/,aaron,hill
7,Abraham Almonte,14343,https://www.spotrac.com/redirect/player/14343/,abraham,almonte
...,...,...,...,...,...
578,Yonder Alonso,7281,https://www.spotrac.com/redirect/player/7281/,yonder,alonso
579,Yorbit Torrealba,276,https://www.spotrac.com/redirect/player/276/,yorbit,torrealba
580,Yoshitomo Tsutsugo,47079,https://www.spotrac.com/redirect/player/47079/,yoshitomo,tsutsugo
581,Yuniesky Betancourt,749,https://www.spotrac.com/redirect/player/749/,yuniesky,betancourt


In [41]:
# create a list of tuples(last, first) to pass into the player lookup function
name_list = []

for i, row in player_search.iterrows():
    tup = (row.last_name, row.first_name)
    name_list.append(tup)
name_list

[('ellis', 'a. j.'),
 ('pierzynski', 'a. j.'),
 ('pollock', 'a. j.'),
 ('hill', 'aaron'),
 ('almonte', 'abraham'),
 ('duvall', 'adam'),
 ('jones', 'adam'),
 ('kennedy', 'adam'),
 ('laroche', 'adam'),
 ('lind', 'adam'),
 ('rosales', 'adam'),
 ('hechavarria', 'adeiny'),
 ('gonzalez', 'adrian'),
 ('almora', 'albert'),
 ('pujols', 'albert'),
 ('callaspo', 'alberto'),
 ('escobar', 'alcides'),
 ('de', 'alejandro'),
 ('avila', 'alex'),
 ('gonzalez', 'alex'),
 ('gordon', 'alex'),
 ('rios', 'alex'),
 ('ramirez', 'alexei'),
 ('amarista', 'alexi'),
 ('simmons', 'andrelton'),
 ('blanco', 'andres'),
 ('torres', 'andres'),
 ('mccutchen', 'andrew'),
 ('jones', 'andruw'),
 ('pagan', 'angel'),
 ('ramirez', 'aramis'),
 ('cabrera', 'asdrubal'),
 ('jackson', 'austin'),
 ('kearns', 'austin'),
 ('romine', 'austin'),
 ('garcia', 'avisail'),
 ('revere', 'ben'),
 ('zobrist', 'ben'),
 ('hall', 'bill'),
 ('butler', 'billy'),
 ('hamilton', 'billy'),
 ('abreu', 'bobby'),
 ('wilson', 'bobby'),
 ('miller', 'brad'),


In [42]:
playerID = player_search_list(name_list)

In [43]:
playerID

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,ellis,a. j.,454560,ellia001,ellisaj01,5677,2008.0,2018.0
1,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
2,hill,aaron,431094,hilla001,hillaa01,6104,2005.0,2017.0
3,almonte,abraham,501659,almoa001,almonab01,5486,2013.0,2021.0
4,duvall,adam,594807,duvaa001,duvalad01,10950,2014.0,2021.0
...,...,...,...,...,...,...,...,...
381,gomes,yan,543228,gomey001,gomesya01,9627,2012.0,2021.0
382,grandal,yasmani,518735,grany001,grandya01,11368,2012.0,2021.0
383,alonso,yonder,475174,alony001,alonsyo01,2530,2010.0,2019.0
384,betancourt,yuniesky,435358,betay001,betanyu01,8585,2005.0,2013.0


In [44]:
# check which entires contain duplicate names
# remove all players whose mlb_played_last <= 2010

pd.set_option('display.max_rows', 10)
playerID[playerID.duplicated(subset=['name_last', 'name_first'], keep=False)]
idx_drop = playerID[playerID['mlb_played_last']<=2010].index
playerID.drop(idx_drop, inplace=True)

In [45]:
# check if there are still duplicated players

playerID[playerID.duplicated(subset=['name_last', 'name_first'], keep=False)]
# drop chris young 3196 and taylor michael 2591, they are not the ones we're looking for
cy_idx = playerID[playerID['key_fangraphs']==3196].index
mt_idx = playerID[playerID['key_fangraphs']==2591].index
playerID.drop(cy_idx, inplace=True)
playerID.drop(mt_idx, inplace=True)

In [46]:
print(f"Number of players we searched: {len(name_list)}")
print(f"Number of result that we got: {len(playerID.index)}")

Number of players we searched: 383
Number of result that we got: 368


In [47]:
# create a list of player name that returned positive results
name_list_positive = []
for i, row in playerID.iterrows():
    tup = (row.name_last, row.name_first)
    name_list_positive.append(tup)
# check who are we still missing
name_list_missing = [x for x in name_list if x not in name_list_positive]
name_list_missing

[('pollock', 'a. j.'),
 ('de', 'alejandro'),
 ('ramirez', 'alexei'),
 ('lemahieu', 'd. j.'),
 ('santana', 'daniel'),
 ('iglesias', 'jose'),
 ('martin', 'leonys'),
 ('pina', 'manuel'),
 ('upton', 'melvin'),
 ('aoki', 'norichika'),
 ('pearce', 'steven'),
 ('la', 'tommy'),
 ('cespedes', 'yoenis'),
 ('torrealba', 'yorbit'),
 ('tsutsugo', 'yoshitomo')]

In [48]:
# create a 'fullname' column in salary_df for merge with playerID

salary_df['full_name'] = salary_df['name'].str.lower().str.replace(' ', '')

# create a fullname column in playerID

playerID['full_name'] = playerID['name_first'] + playerID['name_last']
playerID['full_name'] = playerID['full_name'].str.lower().str.replace(' ', '')

In [50]:
# salary_df.loc[(salary_df['full_name']=='chrisyoung') | (salary_df['full_name']=='michaeltaylor')]

In [51]:
# manualy search these players and append the results to a DF
# hand_searched = pd.DataFrame()

# add1 = playerid_lookup("pollock", "aj", fuzzy=True)
# add1['spotracID'] = 10693

# add2 = playerid_lookup("de aza", "alejandro", fuzzy=True)
# add2['spotracID'] = 7273

# add3 = playerid_lookup("ramirez", "alexei", fuzzy=True).loc[[0]]
# add3['spotracID'] = 5210

# add4 = playerid_lookup("snyder", "chris", fuzzy=True)
# add4['spotracID'] = 127

# add5 = playerid_lookup("lemahieu", "dj", fuzzy=True)
# add5['spotracID'] = 10696

# add6 = playerid_lookup("santana", "daniel", fuzzy=True).loc[[0]]
# add6['spotracID'] = 14323

# add7 = playerid_lookup("iglesias", "jose", fuzzy=True).loc[[0]]
# add7['spotracID'] = 8454

# add8 = playerid_lookup("martin", "leonys", fuzzy=True).loc[[0]]
# add8['spotracID'] = 8627

# add9 = playerid_lookup("pina", "manuel", fuzzy=True).loc[[2]]
# add9['spotracID'] = 8525

# add10 = playerid_lookup("maldonado", "martin", fuzzy=True)
# add10['spotracID'] = 8699

# add11 = playerid_lookup("aoki", "norichika", fuzzy=True).loc[[0]]
# add11['spotracID'] = 8679

# add12 = playerid_lookup("pearce", "steve", fuzzy=True)
# add12['spotracID'] = 7494

# add13 = playerid_lookup("cespedes", "yoenis", fuzzy=True).loc[[0]]
# add13['spotracID'] = 8729

# add14 = playerid_lookup("torrealba", "yorbit", fuzzy=True).loc[[0]]
# add14['spotracID'] = 276

# add15 = playerid_lookup("tsutsugo", "yoshitomo", fuzzy=True).loc[[0]]
# add15['spotracID'] = 47079

# hand_searched.append([add1, add2, add3, add4, add5, add6, add7, add8, add9, add10, add11, add12, add13, add14, add15])
# hand_searched.reset_index(drop=True, inplace=True)